Working with the Detectron Subpackage
-----------------------------------------

This tutorial guides you through the process of setting up and running the Detectron experiment using the `detectron` subpackage, which is designed to assess the robustness of predictive models against covariate shifts in datasets.


### Step 1: Setting up the Datasets
First, configure the `DatasetsManager` to manage the necessary datasets for the experiment. 
To execute the detectron experiment, we need the `training_dataset` used to train the model, alongside the `validation_dataset`. We also need an unseen `reference_dataset` that is driven from the model's knowledge domain, and finally the dataset we want to inspect if it is possibly shifted, the `test_dataset`:


In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from MED3pa.datasets import DatasetsManager

# Initialize the DatasetsManager
datasets = DatasetsManager()

# Load datasets for training, validation, reference, and testing
datasets.set_from_file(dataset_type="training", file='./data/train_data.csv', target_column_name='Outcome')
datasets.set_from_file(dataset_type="validation", file='./data/val_data.csv', target_column_name='Outcome')
datasets.set_from_file(dataset_type="reference", file='./data/test_data.csv', target_column_name='Outcome')
datasets.set_from_file(dataset_type="testing", file='./data/test_data_shifted_0.6.csv', target_column_name='Outcome')


datasets2 = DatasetsManager()

# Load datasets for training, validation, reference, and testing
datasets2.set_from_file(dataset_type="training", file='./data/train_data.csv', target_column_name='Outcome')
datasets2.set_from_file(dataset_type="validation", file='./data/val_data.csv', target_column_name='Outcome')
datasets2.set_from_file(dataset_type="reference", file='./data/test_data.csv', target_column_name='Outcome')
datasets2.set_from_file(dataset_type="testing", file='./data/test_data_shifted_1.6.csv', target_column_name='Outcome')



### Step 2: Configuring the Model
Next, utilize the `ModelFactory` to load a pre-trained model, setting it as the base model manager for the experiment. Alternatively, you can train your own model and use it:


In [2]:
from MED3pa.models import BaseModelManager, ModelFactory

# Initialize the model factory and load the pre-trained model
factory = ModelFactory()
model = factory.create_model_from_pickled("./models/diabetes_xgb_model.pkl")

# Set the base model using BaseModelManager
base_model_manager = BaseModelManager()
base_model_manager.set_base_model(model=model)


## Step 3: Running the Detectron Experiment
Execute the Detectron experiment with the specified datasets and base model. You can also specify other parameters:


In [3]:
from MED3pa.detectron import DetectronExperiment

# Execute the Detectron experiment
experiment_results = DetectronExperiment.run(datasets=datasets, base_model_manager=base_model_manager)
experiment_results2 = DetectronExperiment.run(datasets=datasets2, base_model_manager=base_model_manager)


running seeds: 100%|██████████| 100/100 [00:12<00:00,  7.80it/s]


Detectron execution on reference set completed.


running seeds: 100%|██████████| 100/100 [00:15<00:00,  6.41it/s]


Detectron execution on testing set completed.


running seeds: 100%|██████████| 100/100 [00:14<00:00,  7.09it/s]


Detectron execution on reference set completed.


running seeds: 100%|██████████| 100/100 [00:14<00:00,  6.87it/s]

Detectron execution on testing set completed.


### Step 4: Analyzing the Results
Finally, evaluate the outcomes of the experiment using different strategies to determine the probability of a shift in dataset distributions:


In [4]:
from MED3pa.detectron.strategies import *

# Analyze the results using the disagreement strategies
test_strategies = ["enhanced_disagreement_strategy", "mannwhitney_strategy"]
experiment_results.analyze_results(test_strategies)
experiment_results2.analyze_results(test_strategies)


[{'shift_probability': 0.9444444444444444,
  'test_statistic': 9.722222222222221,
  'baseline_mean': 7.4,
  'baseline_std': 1.2631530214330944,
  'significance_description': {'no shift': 19.641975308641975,
   'small': 14.148148148148149,
   'moderate': 16.283950617283953,
   'large': 49.925925925925924},
  'Strategy': 'enhanced_disagreement_strategy'},
 {'p_value': 4.127692514991661e-13,
  'u_statistic': 2099.5,
  'significance_description': {'no shift': 25.39,
   'small': 11.59,
   'moderate': 29.25,
   'large': 33.77},
  'Strategy': 'mannwhitney_strategy'}]

### Step 5: Saving the Results
You can save the experiment results using the ``save`` method, while specifying the path.

In [5]:
experiment_results.save("./detectron_experiment_results")
experiment_results2.save("./detectron_experiment_results2")


### Step 6: Comparing the Results
You can compare between two detectron experiments using the ``DetectronComparaison`` class, as follows:

In [6]:

from MED3pa.detectron.comparaison import DetectronComparison

comparaison = DetectronComparison("./detectron_experiment_results", "./detectron_experiment_results2")
comparaison.compare_experiments()
comparaison.save("./detectron_experiment_comparaison")